### Switching probability in first 100 trials
- Unstructured sessions showed higher alpha_chosen and lower alpha_perseverance compared to structured sessions, suggesting animals are more likely to switch in the unstructured sessions.

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob()
    print(f"Switch prob: {switch_prob}")

    df = pd.DataFrame(
        {
            "switch_prob": [switch_prob],
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_100trials")

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_100trials

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="grp", y="switch_prob", hue="grp", ax=ax)
sns.stripplot(
    palette=["gray", "gray"],
    edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    # errorbar="se",
    **plot_kw,
)
sns.barplot(
    palette="husl",
    # edgecolor="0.5",
    # facecolor=(0, 0, 0, 0),
    alpha=0.4,
    errorbar="se",
    **plot_kw,
)


orders = ["unstruc", "struc"]
pairs = [(("unstruc"), ("struc"))]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax.set_title("Switch probability")
# ax.legend("")

p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: p <= 5.00e-02

unstruc vs. struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:2.733e-01 Stat=1.200e+00


Text(0.5, 1.0, 'Switch probability')

### Switching probability as a function of trials within session
- Check if switch probability curves for unstructured and structured sessions have same decay and tail

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d


exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob_by_trial()
    switch_prob_smooth = gaussian_filter1d(switch_prob, sigma=1)

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(switch_prob)) + 1,
            "switch_prob": switch_prob,
            "switch_prob_smooth": switch_prob_smooth,
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_by_trial_100trials")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_by_trial_100trials

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob", hue="grp", ax=ax)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)

orders = np.arange(1, 100)
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders[::10]]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax_ = fig.subplot(fig.gs[1])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob_smooth", hue="grp", ax=ax_)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)


ax.set_title("Switch probability")
# ax.legend("")
ax.set_xlim(1, 100)

### Switching probability as per Beron et al. 2022
- It will be interesting to assess switch probability based on history and compare between unstruc and struc.

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []
req_seq = [
    "AAA",
    "aAA",
    "AaA",
    "aaA",
    "AbB",
    "aBB",
    "aBA",
    "ABA",
    "abA",
    "abB",
    "aaB",
    "ABB",
    "AAa",
    "AaB",
    "AbA",
    "aAa",
    "aAB",
    "AAB",
    "Aaa",
    "aBb",
    "ABb",
    "Aba",
    "aba",
    "aaa",
    "aab",
    "aBa",
    "ABa",
    "Aab",
    "abb",
    "AAb",
    "aAb",
    "Abb",
]

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    switch_prob, seq = mab.get_switch_prob_by_history(3)

    # Coding seq of 1,-1,2,-2 to A,a,B,b to be compatible with Beron 2022
    seq = seq.astype(str)
    seq[seq == "-1"] = "a"
    seq[seq == "1"] = "A"
    seq[seq == "-2"] = "b"
    seq[seq == "2"] = "B"

    # Sorting index by most recent reward, then by second most recent reward
    sort_indx = np.lexsort((seq[:, 1], seq[:, 2]))

    # joining to seq into continuous string
    seq = np.array(["".join(map(str, _)) for _ in seq])

    df = pd.DataFrame(
        {
            "switch_prob": switch_prob[sort_indx],
            "seq": seq[sort_indx],
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_seq")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from banditpy.stats import monte_carlo_bootstrap_test
from statplot_utils import stat_kw
from functools import partial

fig = plotting.Fig(1, 1, size=(7, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_seq

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="seq", y="switch_prob", hue="grp", ax=ax)
sns.barplot(
    palette="husl",
    # facecolor=(0, 0, 0, 0),
    alpha=0.5,
    dodge=False,
    # errorbar="se",
    **plot_kw,
)

orders = seq.copy()
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)


def mc_test(x, y):
    return monte_carlo_bootstrap_test(x, y, statistic=np.mean, n_samples=10000)


custom_test = StatTest(mc_test, "MCarlo", "MCarlo")
annotator.configure(test=custom_test, **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax.set_title("Switch probability")
ax.tick_params("x", rotation=90)
ax.set_ylabel("P(Switch)")
ax.set_xlabel("History sequence")
# ax.set_ylim(0, 1)
# ax.legend("")

### SP as per Beron et al. 2022
- Only considering first experience or naive experience of respective environments

In [10]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.first_exposure + mab_subjects.struc.first_exposure

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    switch_prob, seq = mab.get_switch_prob_by_history(3)

    # Coding seq of 1,-1,2,-2 to A,a,B,b to be compatible with Beron 2022
    seq = seq.astype(str)
    seq[seq == "-1"] = "a"
    seq[seq == "1"] = "A"
    seq[seq == "-2"] = "b"
    seq[seq == "2"] = "B"

    # Sorting index by most recent reward, then by second most recent reward
    sort_indx = np.lexsort((seq[:, 1], seq[:, 2]))

    # joining to seq into continuous string
    seq = np.array(["".join(map(str, _)) for _ in seq])

    df = pd.DataFrame(
        {
            "switch_prob": switch_prob[sort_indx],
            "seq": seq[sort_indx],
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_seq_first_exposure")

AggroExp1UnStructured
AuromaExp1Unstructured
BratExp1Unstructured
GrumpExp1Unstructured
BewilderbeastExp1Structured
BuffalordExp1Structured
GronckleExp1Structured
ToothlessExp1Structured
switch_prob_seq_first_exposure saved


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from banditpy.stats import monte_carlo_bootstrap_test
from statplot_utils import stat_kw
from functools import partial

fig = plotting.Fig(1, 1, size=(7, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_seq

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="seq", y="switch_prob", hue="grp", ax=ax)
sns.barplot(
    palette="husl",
    # facecolor=(0, 0, 0, 0),
    alpha=0.5,
    dodge=False,
    # errorbar="se",
    **plot_kw,
)

orders = seq.copy()
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)


def mc_test(x, y):
    return monte_carlo_bootstrap_test(x, y, statistic=np.mean, n_samples=10000)


custom_test = StatTest(mc_test, "MCarlo", "MCarlo")
annotator.configure(test=custom_test, **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax.set_title("Switch probability")
ax.tick_params("x", rotation=90)
ax.set_ylabel("P(Switch)")
ax.set_xlabel("History sequence")
# ax.set_ylim(0, 1)
# ax.legend("")